In [1]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from tqdm import tqdm
from pdf_ingestor import extract_and_split_pdf
from ytb_ingestor import extract_and_split_ytb
from vectorstore import index_documents, index_documents_v2, create_vectorstore
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
import os 
from logger import get_logger
logger = get_logger()

c:\Users\habou\miniconda3\envs\genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embedding_model = HuggingFaceBgeEmbeddings(
    model_name="../../bge-m3",
    encode_kwargs={"normalize_embeddings": True},
    query_instruction="query: "
)

C:\Users\habou\AppData\Local\Temp\ipykernel_25636\961953646.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(


In [ ]:
"""index_path = "../../vectorstore/faiss_vectorestore"
create_vectorstore(embedding_model, index_path)"""

[20:14:17.946] INFO - Création d'un index FAISS vide (dim = 1024)
[20:14:18.053] INFO - Sauvegarde de l'index vide dans: ../../vectorstore/faiss_vectorestore_v2
[20:14:18.055] INFO - Index vide (cosinus) créé avec succès.


In [3]:

dict_ytb_videos_urls = {
    "PLAN STRATEGIQUE RENAULUTION" : "https://www.youtube.com/watch?app=desktop&v=EtivAvmDr2Q&t=901s",
    "Conférence résultats financiers 2021 de Renault Group" : "https://www.youtube.com/watch?v=VfIeaIFSCQA",
    "Conférence résultats financiers 2022 de Renault Group" : "https://www.youtube.com/watch?v=UWHlyjVtwT8",
    "Conférence résultats financiers 2023 de Renault Group" : "https://www.youtube.com/watch?v=B57wephix-w",
    "Conférence résultats financiers 2024 de Renault Group" : "https://www.youtube.com/watch?v=BA5ZOtWfpY0",
}

In [ ]:
"""pdf_chunks = []
ytb_chunks = []
CHUNK_SIZE = 450
OVERLAP = 64

for filename in (pdf_files_list) : 
    file_path = os.path.join(folder_path, filename)
    pdf_chunks.extend(extract_and_split_pdf(file_path, CHUNK_SIZE, OVERLAP))

for video_name, url in dict_ytb_videos_urls.items() : 
    ytb_chunks.extend(extract_and_split_ytb(url, video_name, CHUNK_SIZE, OVERLAP))

all_chunks = pdf_chunks + ytb_chunks"""

'pdf_chunks = []\nytb_chunks = []\nCHUNK_SIZE = 450\nOVERLAP = 64\n\nfor filename in (pdf_files_list) : \n    file_path = os.path.join(folder_path, filename)\n    pdf_chunks.extend(extract_and_split_pdf(file_path, CHUNK_SIZE, OVERLAP))\n\nfor video_name, url in dict_ytb_videos_urls.items() : \n    ytb_chunks.extend(extract_and_split_ytb(url, video_name, CHUNK_SIZE, OVERLAP))\n\nall_chunks = pdf_chunks + ytb_chunks'

### Ingestion PDF

In [ ]:
"""CHUNK_SIZE = 150
OVERLAP = 25
folder_path = "../../data"
markdown_folder = "../../data/md_extracted"
pdf_files_list = ["Renault DEU 2020.pdf", "Renault DEU 2021.pdf", "Renault URD 2022.pdf", "Renault URD 2023.pdf", "Renault 2024 Rapport d’activité.pdf"]
pdf_chunks = []"""

In [ ]:
"""for filename in (pdf_files_list) : 
    file_path = os.path.join(folder_path, filename)
    md_file_path = os.path.join(markdown_folder, filename.split('.')[0]+'.md')
    chunks = extract_and_split_pdf(file_path, md_file_path, CHUNK_SIZE, OVERLAP)
    pdf_chunks.extend(chunks)"""

In [ ]:
"""index_documents(pdf_chunks, embedding_model, "../../vectorstore/faiss_vectorestore")"""

### Ingestion Youtube transcript

In [ ]:
ytb_chunks = []
CHUNK_SIZE = 150
OVERLAP = 25

for video_name, url in dict_ytb_videos_urls.items() : 
    ytb_chunks.extend(extract_and_split_ytb(url, video_name, CHUNK_SIZE, OVERLAP))

In [5]:
ytb_chunks

[]

In [ ]:
"""import pickle

with open("../../vectorstore/documents_v2.pkl", "wb") as f:
    pickle.dump(final_chunks, f)"""

In [ ]:
"""vectorstore = FAISS.load_local(index_path, embedding_model, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()
retriever.get_relevant_documents("(Board of Management - BoM)", k=10)"""

In [ ]:
"""lexical_retriever = BM25Retriever.from_documents(pdf_chunks)
lexical_retriever.k = 3
lexical_retriever.get_relevant_documents("Identify the members of the board of directors in 2021.")"""